In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!fairfaceequ ls

/bin/bash: fairfaceequ: command not found


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
!unzip fairfaceretinex.zip

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time

import os
import random
import numpy as np

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transforms = transforms.Compose([
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomRotation(10),
                           transforms.RandomCrop((224, 224), pad_if_needed=True),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                       ])

test_transforms = transforms.Compose([
                           transforms.CenterCrop((224, 224)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                       ])

In [ ]:
train_data = datasets.ImageFolder('fairfaceretinex/Training', train_transforms)
valid_data = datasets.ImageFolder('fairfaceretinex/Validation', test_transforms)
#test_data = datasets.ImageFolder('fairfaceequ/Validation/Male/1.jpg', test_transforms)

#import os

#print(len(os.listdir('data/dogs-vs-cats/train')))

#n_train_examples = int(len(train_data)*0.9)
#n_valid_examples = n_test_examples = len(train_data) - n_train_examples

#train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples])
#train_data, test_data = torch.utils.data.random_split(train_data, [n_train_examples-n_valid_examples, n_test_examples])

In [ ]:
print(valid_data.class_to_idx)

{'Female': 0, 'Male': 1}


https://github.com/facebook/fb.resnet.torch/issues/180
https://github.com/bamos/densenet.pytorch/blob/master/compute-cifar10-mean.py

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
#print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 85902
Number of validation examples: 10836


In [ ]:
BATCH_SIZE = 64

train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
#test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

https://discuss.pytorch.org/t/why-does-the-resnet-model-given-by-pytorch-omit-biases-from-the-convolutional-layer/10990/4
https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

In [ ]:
device = torch.device('cuda')

In [ ]:
import torchvision.models as models

model = models.resnet18(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
print(model)

In [ ]:
from torch.optim import lr_scheduler


num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Adam optimizer 

In [ ]:
from torch.optim import lr_scheduler


num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999),eps=1e-08, weight_decay=0, amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [ ]:
def train(model, device, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, device, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

old model without history

In [ ]:
EPOCHS = 60
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'RaceMultiscaleretinex-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

| Epoch: 01 | Train Loss: 0.578 | Train Acc: 68.48% | Val. Loss: 0.512 | Val. Acc: 73.78% |
| Epoch: 02 | Train Loss: 0.500 | Train Acc: 74.05% | Val. Loss: 0.441 | Val. Acc: 78.10% |
| Epoch: 03 | Train Loss: 0.471 | Train Acc: 75.90% | Val. Loss: 0.413 | Val. Acc: 79.58% |
| Epoch: 04 | Train Loss: 0.455 | Train Acc: 76.98% | Val. Loss: 0.396 | Val. Acc: 80.34% |
| Epoch: 05 | Train Loss: 0.442 | Train Acc: 77.81% | Val. Loss: 0.379 | Val. Acc: 81.55% |
| Epoch: 06 | Train Loss: 0.433 | Train Acc: 78.28% | Val. Loss: 0.385 | Val. Acc: 81.26% |
| Epoch: 07 | Train Loss: 0.425 | Train Acc: 79.02% | Val. Loss: 0.396 | Val. Acc: 80.33% |
| Epoch: 08 | Train Loss: 0.418 | Train Acc: 79.31% | Val. Loss: 0.363 | Val. Acc: 82.32% |
| Epoch: 09 | Train Loss: 0.413 | Train Acc: 79.51% | Val. Loss: 0.354 | Val. Acc: 82.78% |
| Epoch: 10 | Train Loss: 0.405 | Train Acc: 80.15% | Val. Loss: 0.348 | Val. Acc: 83.58% |
| Epoch: 11 | Train Loss: 0.403 | Train Acc: 80.08% | Val. Loss: 0.366 | Val. Ac

In [ ]:
Adam epoch increment model

epoch increment history model

In [ ]:
EPOCHS = 60
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ColabRaceresnet-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')

| Epoch: 01 | Train Loss: 0.573 | Train Acc: 68.73% | Val. Loss: 0.532 | Val. Acc: 72.60% | Time: 946.2113s |
| Epoch: 02 | Train Loss: 0.498 | Train Acc: 74.32% | Val. Loss: 0.428 | Val. Acc: 78.78% | Time: 910.7252s |
| Epoch: 03 | Train Loss: 0.469 | Train Acc: 76.22% | Val. Loss: 0.404 | Val. Acc: 80.06% | Time: 900.1948s |
| Epoch: 04 | Train Loss: 0.455 | Train Acc: 76.83% | Val. Loss: 0.407 | Val. Acc: 80.38% | Time: 892.6625s |
| Epoch: 05 | Train Loss: 0.442 | Train Acc: 77.81% | Val. Loss: 0.388 | Val. Acc: 81.05% | Time: 882.2377s |
| Epoch: 06 | Train Loss: 0.435 | Train Acc: 78.39% | Val. Loss: 0.439 | Val. Acc: 78.77% | Time: 869.0003s |
| Epoch: 07 | Train Loss: 0.425 | Train Acc: 78.89% | Val. Loss: 0.363 | Val. Acc: 82.69% | Time: 859.3664s |
| Epoch: 08 | Train Loss: 0.418 | Train Acc: 79.46% | Val. Loss: 0.362 | Val. Acc: 82.27% | Time: 859.3354s |
| Epoch: 09 | Train Loss: 0.410 | Train Acc: 79.63% | Val. Loss: 0.373 | Val. Acc: 82.12% | Time: 899.7110s |
| Epoch: 1

In [ ]:
EPOCHS = 100
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'ClbRaceresnet-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')

| Epoch: 01 | Train Loss: 0.573 | Train Acc: 68.73% | Val. Loss: 0.532 | Val. Acc: 72.60% | Time: 905.4446s |
| Epoch: 02 | Train Loss: 0.498 | Train Acc: 74.32% | Val. Loss: 0.428 | Val. Acc: 78.78% | Time: 885.2020s |
| Epoch: 03 | Train Loss: 0.469 | Train Acc: 76.22% | Val. Loss: 0.404 | Val. Acc: 80.06% | Time: 868.2649s |
| Epoch: 04 | Train Loss: 0.455 | Train Acc: 76.83% | Val. Loss: 0.407 | Val. Acc: 80.38% | Time: 895.0608s |
| Epoch: 05 | Train Loss: 0.442 | Train Acc: 77.81% | Val. Loss: 0.388 | Val. Acc: 81.05% | Time: 871.4615s |
| Epoch: 06 | Train Loss: 0.435 | Train Acc: 78.39% | Val. Loss: 0.439 | Val. Acc: 78.77% | Time: 837.8308s |
| Epoch: 07 | Train Loss: 0.425 | Train Acc: 78.89% | Val. Loss: 0.363 | Val. Acc: 82.69% | Time: 839.9215s |
| Epoch: 08 | Train Loss: 0.418 | Train Acc: 79.46% | Val. Loss: 0.362 | Val. Acc: 82.27% | Time: 859.4010s |
| Epoch: 09 | Train Loss: 0.410 | Train Acc: 79.63% | Val. Loss: 0.373 | Val. Acc: 82.12% | Time: 835.1146s |
| Epoch: 1

In [ ]:
valid_data = datasets.ImageFolder('fairfaceretinex/Validation', test_transforms)


valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)
model.load_state_dict(torch.load('models/RaceMultiscaleretinex-ResentFT-males-vs-females.pt'))

test_loss, test_acc = evaluate(model, device, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')

| Test Loss: 0.316 | Test Acc: 85.34% |


In [ ]:
import matplotlib.pyplot as plt
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,10)
plt.savefig('loss_curve.png')
plt.show()

In [ ]:
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig('accuracy_curve.png')
plt.show()

In [ ]:
print(optimizer.state_dict())

f = open( 'test_optimizer_state_dict.txt', 'w' )
f.write( 'test_optimizer_state_dict = ' + repr(optimizer.state_dict()) + '\n' )
f.close()

In [ ]:
#print(model.state_dict())

f = open( 'test_model_state_dict.txt', 'w' )
f.write( 'test_model_state_dict = ' + repr(model.state_dict()) + '\n' )
f.close()

In [ ]:
dict = {'one': 1, 'two': 2}
f = open( 'test123.txt', 'w' )
f.write( 'dict = ' + repr(dict) + '\n' )
f.close()

In [ ]:
print(history[:,3:4])
flat_list = [item for sublist in history[:,3:4] for item in sublist]

[[0.84419118]]


In [ ]:
best_acc=max(flat_list)
print(best_acc)

0.8441911764004651


In [ ]:
#trial1 = torch.load('epoch_60.pt')
#print(trial1)
#print(model)
import shutil
def save_checkpoint123(state123, is_best123, filename123='checkpoint.pth.tar'):
    torch.save(state123, filename123)
    if is_best123:
        shutil.copyfile(filename123, 'model_best.pth.tar')
        
save_checkpoint123({
            'epoch': epoch + 1,
            'arch': 'resnet18',
            'state_dict': model.state_dict(),
            'best_prec1': best_acc,
            'optimizer' : optimizer.state_dict(),
            'all_history': history,
            'exp_lr_scheduler_dict' : exp_lr_scheduler.state_dict(),
        }, True)

In [ ]:
resume = 'model_best.pth.tar'
if resume:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            all_history = (checkpoint['all_history']).tolist()
            exp_lr_scheduler = checkpoint['exp_lr_scheduler_dict']
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 1)


In [ ]:
print(start_epoch)
print(all_history)
history = all_history

In [ ]:
EPOCHS = 4
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'EpochIncrementRaceresnet-ResentFT-males-vs-females.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
history = []
for epoch in range(start_epoch,start_epoch+EPOCHS):
    epoch_start = time.time()
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    history.append([train_loss, valid_loss, train_acc, valid_acc])
    epoch_end = time.time()
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% | Time: {epoch_end-epoch_start:.4f}s |')
    

| Epoch: 01 | Train Loss: 0.345 | Train Acc: 83.44% | Val. Loss: 0.332 | Val. Acc: 84.42% | Time: 1045.6672s |


In [ ]:
model.load_state_dict(torch.load('models/EpochIncrementRaceresnet-ResentFT-males-vs-females.pt'))
test_data = datasets.ImageFolder(dataset+'\\validation', test_transforms)
class_names = test_data.classes

dataloaders = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
nb_classes = 460


confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix)
print(confusion_matrix.diag()/confusion_matrix.sum(1))
Ind_acc = confusion_matrix.diag()/confusion_matrix.sum(1)
print("Female : ", float(Ind_acc[0]))s
print("Male : ", float(Ind_acc[1]))
TN = int(confusion_matrix[0][0])
FN = int(confusion_matrix[0][1])
FP = int(confusion_matrix[1][0])
TP = int(confusion_matrix[1][1])

In [ ]:
FPR = 0
FNR = 0
TPR = 0
TNR = 0
FPR = FP/(FP + TN)
FNR = FN/(FN + TP)
TPR = 1 - FNR
TNR = 1 - FPR
Overall_acc = (TPR + TNR)/2
print("Overall Accuracy : ", Overall_acc)
print("Female : ", float(Ind_acc[0]))
print("Male : ", float(Ind_acc[1]))
print("FPR : ", FNR)
print("FNR : ", FPR)

In [ ]:
   model.eval()
    num_correct = 0 
    num_examples = 0
    for batch in val_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        output = model(inputs)
        targets = targets.to(device)
        loss = loss_fn(output,targets) 
        valid_loss += loss.data.item() * inputs.size(0)
        correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    valid_loss /= len(val_loader.dataset)
 
    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
    valid_loss, num_correct / num_examples))

In [ ]:
summary(model)

end of epoch increment

In [ ]:
#val = r'../pradeep/Desktop/FinalRacedataets/RaceEqu/Validation/'
val = 'Raceretinex/Validation'
races = os.listdir(val)
genders = ['Male','Female']
for race in races:
    for gender in genders:
        finalPath = val+race+'/'+gender
        #print(finalPath)


        test_data = datasets.ImageFolder(finalPath, test_transforms)


        test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
        model.load_state_dict(torch.load('models/ResentFT-males-vs-females.pt'))

        test_loss, test_acc = evaluate(model, device, test_iterator, criterion)

        print(f'{race} {gender}| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')


In [ ]:
data_transforms = {
    'test': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['test', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['6-10', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['6-10', 'val']}
class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model.load_state_dict(torch.load('models/ResentFT-males-vs-females.pt'))
test_data = datasets.ImageFolder('fairfaceretinex/Validation', test_transforms)
class_names = test_data.classes

dataloaders = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
nb_classes = 2


confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

tensor([[4559.,  545.],
        [ 568., 5164.]])


In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.8932, 0.9009])


In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion() 

def visualize_model(model, num_images=22):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
           # print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(22)))

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                print('GroudthTruth: ',class_names[labels[j]])
                ax.set_title('Predicted: {}'.format(class_names[preds[j]]))
               
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
out = torchvision.utils.make_grid(inputs)

visualize_model(model)
plt.ioff()
plt.show()